In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

# TCRD Intro (Appyter)

* http://juniper.health.unm.edu/tcrd/
* http://juniper.health.unm.edu/tcrd/api.html
* https://pharos.nih.gov/idg/api

In [2]:
import sys, json, urllib.request
import pandas as pd
import matplotlib.pyplot
import mysql.connector as mysql

In [3]:
DBHOST = "tcrd.kmc.io"
DBNAME = "tcrd6110"
DBUSR = "tcrd"

In [4]:
dbcon = mysql.connect(host=DBHOST, db=DBNAME, user=DBUSR)

### Db Info

In [8]:
pd.read_sql("SELECT * FROM dbinfo", dbcon)

,dbname,schema_ver,data_ver,owner,is_copy,dump_file
0,tcrd,6.6.2,6.11.0,smathias,0,None


### Select target

In [ ]:
%%appyter code_exec

{% do SectionField(name='target_selection',
        title='Target Selection',
        subtitle='Choose between gene symbol or UniprotId.')%}


{% set target_field = TabField(
    name='target_field',
    label='Target Selection',
    default='Gene symbol',
    description='Select input method',
    choices={
        'Gene symbol': [
            StringField(
                name='input_symbol',
                label='Gene symbol',
                default='CDK1',
                description='Input your gene symbol of interest.',
                section = 'target_selection'
            ),

        ],
        
        'UniprotId': [
                StringField(
                name='input_uniprot',
                label='UniprotId',
                default='P06493',
                description='Input your UniprotId of interest.',
                section = 'target_selection'
            ),
        ],
    },
    section = 'target_selection'
) %}

In [ ]:
%%appyter code_exec

{% set uniprotId, geneSymbol = '', '' %}
{% if target_field.raw_value == 'UniprotId' %}
{% set uniprotId = target_field.value[0] %}
{% else %}
{% set geneSymbol = target_field.value[0] %}
{% endif %}

{% if uniprotId != '' %}

sql = """SELECT * FROM target
LEFT OUTER JOIN t2tc ON t2tc.target_id = target.id
LEFT OUTER JOIN protein p ON p.id = t2tc.protein_id
WHERE p.uniprot = {{uniprotId}}"""

{% else %}

sql = """SELECT * FROM target
LEFT OUTER JOIN t2tc ON t2tc.target_id = target.id
LEFT OUTER JOIN protein p ON p.id = t2tc.protein_id
WHERE p.sym = {{geneSymbol}}"""

{% endif %}

df = pd.read_sql(sql, dbcon)
df